### Чтение файлов .csv

Самый простой способ хранения табличных данных &mdash; файлы .csv. По сути, это всё те же текстовые файлы (plain text). В каждой строке такого файла хранится одна строка таблицы, её ячейки разделены специальным символом-разделителем (delimiter). По умолчанию этот символ &mdash; запятая (именно поэтому формат называется .csv &mdash; comma-separated values). Если в роли разделителя выступает другой символ, могут использоваться другие расширения, например, .tsv (tab-separated values). Файлы .csv поддерживаются всеми табличными редакторами, поэтому их можно открывать в Excel, Google Spreadsheets или любом другом редакторе, которым вы привыкли пользоваться.

Важно помнить, что, поскольку файлы .csv представляют собой текстовые файлы, они также могут быть подвержены проблемам с кодировками. 

Первая строчка таблицы может содержать названия столбцов. В таком случае она называется заголовком (или шапкой) таблицы (header row).

Для работы с файлами .csv в Python существует разные сторонние библиотеки, но самое простое средство &mdash; встроенная библиотека `csv`. Она поддерживает чтение и запись в разных режимах и с разными разделителями.

In [ ]:
!wget https://phonetics-spbu.github.io/courses/python_ai_mag/files/work_types.csv
!wget https://phonetics-spbu.github.io/courses/python_ai_mag/files/student_marks.csv

Функция `reader()` возвращает специальный объект для чтения csv-файла. Он является итерируемым объектом и на каждом шаге цикла выдаёт список строк, соответствующих ячейкам таблицы в одной строчке. Обратите внимание на аргумент `newline=""` при открытии файла: он нужен для того, чтобы модуль `csv` корректно обрабатывал переводы строки независимо от платформы.

In [ ]:
import csv

with open("work_types.csv", newline="", encoding="utf-8") as f:
    reader = csv.reader(f)
    for row in reader:
        print(row)

['работа на уроке', '1.0']
['домашняя работа', '1.0']
['самостоятельная работа', '1.2']
['практическая работа', '1.3']
['проверочная работа', '1.3']
['контрольная работа', '1.5']


Его можно переделать в список, тогда весь файл прочитается в память и доступ к данным можно будет получить вне контекстного менеджера (то есть вне `with`).

In [ ]:
import csv

with open("work_types.csv", newline="", encoding="utf-8") as f:
    reader = csv.reader(f)
    work_types_rows = list(reader)

print(work_types_rows)

[['работа на уроке', '1.0'], ['домашняя работа', '1.0'], ['самостоятельная работа', '1.2'], ['практическая работа', '1.3'], ['проверочная работа', '1.3'], ['контрольная работа', '1.5']]


Обратите внимание, что любые данные (текстовые и числовые) возвращаются в виде строк. Чтобы работать с числовыми данными как с числами, нужно привести их к нужному типу.

**Контрольный вопрос:** как из такого списка сделать словарь? Пусть его значениями будут вещественные числа.

<details>
<summary>Ответ:</summary>
<pre>
work_types = dict(work_types_rows)
for k, v in work_types.items():
    work_types[k] = float(v)
# или
work_types = {}
for k, v in work_types_rows:
    work_types[k] = float(v)
</pre>
</details>

Для записи данных в файл существует функция `writer()`. Она возвращает специальный объект для записи, у которого есть методы `.writerow()` для записи одной строчки таблицы (он принимает на вход список значений) и `.writerows()` для записи нескольких строк сразу (он принимает на вход список списков значений). Значения могут быть как строками, так и другими значениями (они будут превращены в строки с помощью `str()`).

In [ ]:
work_types_rows.append(["программирование", 2.0])

with open("work_types_mod.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(work_types_rows)

Если файл как-то отличается от конфигурации по умолчанию, можно прописать эти различия в специальных аргументах к функциям `reader` и `writer`:

* `delimiter` &mdash; разделитель: например, `";"` или `"\t"`. По умолчанию &mdash; `","`.
* `quotechar` &mdash; кавычки, в которые заключаются строки, в которых есть специальные символы (например, `delimiter`). По умолчанию &mdash; `'"'`.
* `escapechar` &mdash; символ, позволяющий экранировать символы (то есть делать их не специальными, а обычными). По умолчанию &mdash; `None`.
* `doublequote` &mdash; логическое значение, показывающее, что делать с `quotechar` в середине поля. Если `True`, то он будет удваиваться. По умолчанию &mdash; `True`.

С полным списком можно ознакомиться в [документации](https://docs.python.org/3/library/csv.html#csv-fmt-params).

Предустановленные наборы параметров заключены в т.н. &laquo;диалектах&raquo;. Чтобы посмотреть, какие есть диалекты, воспользуемся функциями `list_dialects()` и `get_dialect()`.

In [22]:
for name in csv.list_dialects():
    dialect = csv.get_dialect(name)
    print((name, dialect.delimiter, dialect.lineterminator))

('excel', ',', '\r\n')
('excel-tab', '\t', '\r\n')
('unix', ',', '\n')


Название диалекта передаётся в качестве аргумента в функции `reader()` и `writer()`: 

In [ ]:
with open("work_types.csv", newline="", encoding="utf-8") as f:
    reader = csv.reader(f, dialect="excel")
    work_types_rows = list(reader)

print(work_types_rows)

Если у таблицы есть шапка, то можно воспользоваться специальными классами `DictReader` и `DictWriter`. В этом случае каждая строчка таблицы будет представлена не в виде списка значений, а в виде словаря, ключами которого будут названия столбцов, а значениями &mdash; строки с данными из соответствующих ячеек. 

In [ ]:
with open("student_marks.csv", newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    student_marks = list(reader)
    header = reader.fieldnames

print(header)
for row in student_marks[:10]:
    print(row)

['фамилия', 'имя', 'предмет', 'вид работы', 'оценка']
{'фамилия': 'Голубева', 'имя': 'Марина', 'предмет': 'французский язык', 'вид работы': 'проверочная работа', 'оценка': '4'}
{'фамилия': 'Ветровская', 'имя': 'Анастасия', 'предмет': 'фонетика', 'вид работы': 'практическая работа', 'оценка': '3'}
{'фамилия': 'Звездин', 'имя': 'Артемий', 'предмет': 'информатика', 'вид работы': 'работа на уроке', 'оценка': '5'}
{'фамилия': 'Ветровская', 'имя': 'Анастасия', 'предмет': 'информатика', 'вид работы': 'практическая работа', 'оценка': '4'}
{'фамилия': 'Солодовников', 'имя': 'Виктор', 'предмет': 'французский язык', 'вид работы': 'контрольная работа', 'оценка': '5'}
{'фамилия': 'Солодовников', 'имя': 'Виктор', 'предмет': 'информатика', 'вид работы': 'практическая работа', 'оценка': '2'}
{'фамилия': 'Воронцова', 'имя': 'Кира', 'предмет': 'математика', 'вид работы': 'работа на уроке', 'оценка': '5'}
{'фамилия': 'Громов', 'имя': 'Илья', 'предмет': 'французский язык', 'вид работы': 'домашняя работа',

**Контрольный вопрос:** как отфильтровать строчки таблицы с нужными значениями? Например, оставим в списке только оценки за контрольные работы.

<details>
<summary>Ответ:</summary>
<pre>
test_marks = [row for row in student_marks if row["вид работы"] == "контрольная работа"]
for row in test_marks:
    print(row)
</pre>
</details>

**Контрольный вопрос:** как теперь вычислить среднюю оценку за контрольные работы?

3.7


<details>
<summary>Ответ:</summary>
<pre>
avg = sum([int(row["оценка"]) for row in test_marks]) / len(test_marks)
print(avg)
</pre>
</details>

Если шапки нет, но названия полей мы знаем заранее, их можно передать в аргумент `fieldnames`:

In [ ]:
with open("work_types.csv", newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f, fieldnames=["тип работы", "коэффициент"])
    rows = list(reader)
    header = reader.fieldnames

print(header)
for row in rows:
    print(row)

['тип работы', 'коэффициент']
{'тип работы': 'работа на уроке', 'коэффициент': '1.0'}
{'тип работы': 'домашняя работа', 'коэффициент': '1.0'}
{'тип работы': 'самостоятельная работа', 'коэффициент': '1.2'}
{'тип работы': 'практическая работа', 'коэффициент': '1.3'}
{'тип работы': 'проверочная работа', 'коэффициент': '1.3'}
{'тип работы': 'контрольная работа', 'коэффициент': '1.5'}


`DictWriter` работает аналогично `writer`, но требует два аргумента, второй из которых &mdash; список названий столбцов. Чтобы записать их в файл, нужно вызвать метод `.writeheader()`.

In [35]:
with open("student_marks.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=header)
    writer.writeheader()
    writer.writerows(rows)

### Сортировка: продвинутый уровень

Чтобы задать сложный принцип сортировки, можно использовать т.н. лямбда-функции:

In [36]:
a = [1, 3, 4, 2, 6, 7]
print(sorted(a, key=lambda x: x ** 2 - 5 * x))

[3, 2, 1, 4, 6, 7]


Синтаксис такой:

`key=lambda x: выражение с x, по которому будем сравнивать сортируемые элементы`

Вместо `x` можно использовать любое другое имя.

In [37]:
rows = [
    ["John", 20, 170], # имя, возраст, рост
    ["Jane", 24, 175],
    ["Jill", 22, 177],
]
print(sorted(rows, key=lambda x: x[0])) # сортируем по имени
print(sorted(rows, key=lambda x: x[1])) # сортируем по возрасту
print(sorted(rows, key=lambda x: x[2])) # сортируем по росту

[['Jane', 24, 175], ['Jill', 22, 177], ['John', 20, 170]]
[['John', 20, 170], ['Jill', 22, 177], ['Jane', 24, 175]]
[['John', 20, 170], ['Jane', 24, 175], ['Jill', 22, 177]]


Если табличные данные заданы как список словарей:

In [38]:
rows = [
    {"name": "John", "age": 20, "height": 170},
    {"name": "Jane", "age": 24, "height": 175},
    {"name": "Jill", "age": 22, "height": 177},
]
print(sorted(rows, key=lambda x: x["name"]))
print(sorted(rows, key=lambda x: x["age"]))
print(sorted(rows, key=lambda x: x["height"]))

[{'name': 'Jane', 'age': 24, 'height': 175}, {'name': 'Jill', 'age': 22, 'height': 177}, {'name': 'John', 'age': 20, 'height': 170}]
[{'name': 'John', 'age': 20, 'height': 170}, {'name': 'Jill', 'age': 22, 'height': 177}, {'name': 'Jane', 'age': 24, 'height': 175}]
[{'name': 'John', 'age': 20, 'height': 170}, {'name': 'Jane', 'age': 24, 'height': 175}, {'name': 'Jill', 'age': 22, 'height': 177}]


Сортировка по двум параметрам (сначала по одному, потом по другому):

In [43]:
rows = [
    {"name": "John", "age": 20, "height": 170},
    {"name": "Jane", "age": 24, "height": 175},
    {"name": "Jill", "age": 22, "height": 175},
    {"name": "Jack", "age": 22, "height": 177},
]
print(sorted(rows, key=lambda x: (x["age"], x["height"])))

[{'name': 'John', 'age': 20, 'height': 170}, {'name': 'Jill', 'age': 22, 'height': 175}, {'name': 'Jack', 'age': 22, 'height': 177}, {'name': 'Jane', 'age': 24, 'height': 175}]


Вспомним, что параметр `key` можно передавать и в функции `min()` и `max()`:

In [41]:
rows = [
    {"name": "John", "age": 20, "height": 170},
    {"name": "Jane", "age": 24, "height": 175},
    {"name": "Jill", "age": 22, "height": 177},
]
print(max(rows, key=lambda x: x["height"]))

{'name': 'Jill', 'age': 22, 'height': 177}


**Контрольный вопрос:** как отсортировать строчки таблицы `student_marks`, чтобы все виды работ шли в алфавитном порядке, а внутри каждого вида работы &mdash; оценки шли по убыванию?

<details>
<summary>Ответ:</summary>
<pre>
student_marks = sorted(student_marks, key=lambda x: (x["вид работы"], -int(x["оценка"])))
for row in student_marks[:15]:
    print(row)
</pre>
</details>

### Практические задания

#### Задание 1

По таблице *student_marks.csv* определите количество учеников в классе и составьте алфавитный список.

#### Задание 2

а) для каждого ученика определите средний балл по каждому предмету.

б) определите средневзвешенную оценку по каждому предмету с использованием весов из таблицы *work_types.csv*.

в) составьте таблицу с этими данными и запишите её в отдельный файл .csv. Первый столбец таблицы пусть содержит фамилию и имя, остальные &mdash; средневзвешенную оценку по каждому предмету в алфавитном порядке.

#### Задание 3

Найдите ученика с максимальным средним баллом по информатике.

#### Задание 4

Файл *messages.csv* содержит данные о сообщениях в некотором чате. Доступна информация: имя пользователя, дата сообщения, количество слов. Найдите среднее, максимальное и минимальное количество слов в сообщениях каждого пользователя. Найдите пользователя с самым длинным сообщением, отправленным после 18:00.

#### Задание 5 (*)

Напишите свою программу для чтения файла .csv с заголовком (без учёта кавычек, экранирования и прочего).

### Домашнее задание

Используя файл *result.json* с сообщениями канала ОКСФ, составьте таблицу со следующими данными: дата, время, длина сообщения в символах (то есть сумма длин всех полей `"text"` в словаре `"text_entities"`), есть ли прикреплённая картинка, количество реакций. Запишите её в файл .csv.